# Rossmann Store Sales Prediction

I'll implement a XGBoost model and a Nerual Network model with embedding.

**Key words:**

- XGBoost, One-Hot Encoding, Target Encoding, Pipeline, Cross-Validation

## Import and Set Up

In [ ]:
!pip install category_encoders

In [ ]:
# Import packages
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import make_scorer
from category_encoders import TargetEncoder
from xgboost import XGBRegressor
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
import joblib

print("Import complete!")

Import complete!


In [ ]:
# Set up global variables
drive.mount('/content/drive')

# Dataset paths
TRAIN_PATH = '/content/drive/MyDrive/data-science-notebooks/rossmann-store-sales-prediction/datasets/train.csv'
TEST_PATH = '/content/drive/MyDrive/data-science-notebooks/rossmann-store-sales-prediction/datasets/test.csv'
MODELS_PATH = '/content/drive/MyDrive/data-science-notebooks/rossmann-store-sales-prediction/models/'

print("Set up complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Set up complete!


## Dataset

Rossmann Store Sales

This dataset is from a competition on Kaggle.

Link: https://www.kaggle.com/competitions/rossmann-store-sales

It contains the information of drug stores. It's used to predict 6 weeks of daily sales for 1,115 stores located across Germany.

I will take a glance at training set and test set

In [ ]:
# Training set
df = pd.read_csv(TRAIN_PATH)
df.info()

/tmp/ipython-input-2703899870.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(TRAIN_PATH)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [ ]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


**Feature Explanation**

- Store - a unique Id for each store

- DayOfWeek - a day of the week

- Date - the date

- Sales - the turnover for any given day (this is what I'm predicting)

- Customers - the number of customers on a given day

- Open - an indicator for whether the store was open: 0 = closed, 1 = open

- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None

- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools

Take a glance at the competition test data. It won't be used during model training and validation. It's for scoring purposes in the competition.

In [ ]:
df_test = pd.read_csv(TEST_PATH)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             41088 non-null  int64  
 1   Store          41088 non-null  int64  
 2   DayOfWeek      41088 non-null  int64  
 3   Date           41088 non-null  object 
 4   Open           41077 non-null  float64
 5   Promo          41088 non-null  int64  
 6   StateHoliday   41088 non-null  object 
 7   SchoolHoliday  41088 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 2.5+ MB


## Preprocessing

Implement the preprocessing that won't cause data leakage.

- Extract time information from column "Date"
- Convert all categorical features to categorical type and replace them with numerical codes. If I don't replace them with numerical codes, the embedding in NN later will not match.
- only keep the features included in test set.

By the way, there is a "store.csv" dataset. It's supplemental information about the stores. I won't use it for now, because the test set doesn't include the features in this dataset.

In [ ]:
df_prep = df.copy()

# Extract information from Date
df_prep['Date'] = pd.to_datetime(df['Date'])
df_prep['Year'] = df_prep['Date'].dt.year
df_prep['Month'] = df_prep['Date'].dt.month
df_prep['Day'] = df_prep['Date'].dt.day
df_prep = df_prep.drop(columns=['Date'])

# Remove the feature that is not in the test set
df_prep = df_prep.drop(columns=['Customers'])

# Split X and y
X = df_prep.drop(columns=['Sales'])
y = df_prep['Sales']

# Transform all columns in X to categorical and replace by numerical codes.
X = X.astype('category')
X = X.apply(lambda x: x.cat.codes)

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype
---  ------         --------------    -----
 0   Store          1017209 non-null  int16
 1   DayOfWeek      1017209 non-null  int8 
 2   Open           1017209 non-null  int8 
 3   Promo          1017209 non-null  int8 
 4   StateHoliday   1017209 non-null  int8 
 5   SchoolHoliday  1017209 non-null  int8 
 6   Year           1017209 non-null  int8 
 7   Month          1017209 non-null  int8 
 8   Day            1017209 non-null  int8 
dtypes: int16(1), int8(8)
memory usage: 9.7 MB


## XGBoost

I'm going to implement a XGBoost model first. Compare to Neural Network models, it's easy to interpret. I'll use a pipeline.

### Column Transformer, Pipeline and RMSPE Metric

Prepare column transformer for pipeline

- Target encoding: 'Store'
- One-Hot encoding: 'DayOfWeek', 'StateHoliday', 'SchoolHoliday' 'Year', 'Month', 'Day'

No normalization, because XGBoost model is not sensitive with the scale.

In [ ]:
# Build a column transformer
preprocessor = ColumnTransformer([
    ('target_encoding', TargetEncoder(cols=['Store']), ['Store']),
    ('one_hot_encoding', OneHotEncoder(handle_unknown='ignore'), ['DayOfWeek', 'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Day'])
])

print("Column transformer completed!")

Column transformer completed!


In [ ]:
# Build a pipeline
pipeline_xgboost = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

print("Pipeline completed!")

Pipeline completed!


Build a RMSPE score metric for cross validation.

- Lower is better
- 0 is the best
- Target score is less than 0.12

In [ ]:
def rmspe(y_true, y_pred):
    # Filter out entries where y_true is zero to avoid division by zero
    non_zero_mask = y_true != 0
    y_true_filtered = y_true[non_zero_mask]
    y_pred_filtered = y_pred[non_zero_mask]

    # Ensure predictions are non-negative
    y_pred_filtered[y_pred_filtered < 0] = 0

    return np.sqrt(np.mean(np.square((y_true_filtered - y_pred_filtered) / y_true_filtered)))

rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

print("RMSPE scorer completed!")

RMSPE scorer completed!


### Model Training and Evaluation with Cross Validation

I'll tune three hyper-parameters:

- max_depth: Controls model complexity. Larger means more complex.
- n_estimators: Controls how long boosting continues.
- learning_rate: Controls each tree's contribution. Works together with n_estimators. Smaller learning rate has better performance if I increase n_estimators.

In [ ]:
# Prepare cross validation
param_grid = {
    'regressor__max_depth': [5, 7, 9],
    'regressor__n_estimators': [500, 1000, 1500],
    'regressor__learning_rate': [0.05, 0.1, 0.3]
}

cv = KFold(n_splits=3, shuffle=True, random_state=936)

grid_search_cv = GridSearchCV(
    estimator=pipeline_xgboost,
    param_grid=param_grid,
    scoring=rmspe_scorer,
    cv=cv,
    verbose=2,
    n_jobs=-1
)

print("Grid search CV is prepared!")

Grid search CV is prepared!


In [ ]:
# Fit the CV
grid_search_cv.fit(X, y)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=KFold(n_splits=3, random_state=936, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('target_encoding',
                                                                         TargetEncoder(cols=['Store']),
                                                                         ['Store']),
                                                                        ('one_hot_encoding',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['DayOfWeek',
                                                                          'StateHoliday',
                                                                          'SchoolHoliday',
                                                                          'Year',
                                                                          'Month',
                                                                          'Day'])])),
                                       ('regressor',
                                        XGBRegress...
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None, ...))]),
             n_jobs=-1,
             param_grid={'regressor__learning_rate': [0.05, 0.1, 0.3],
                         'regressor__max_depth': [5, 7, 9],
                         'regressor__n_estimators': [500, 1000, 1500]},
             scoring=make_scorer(rmspe, greater_is_better=False, response_method='predict'),
             verbose=2)

In [ ]:
# Show the result
print("Best parameters:", grid_search_cv.best_params_)
print("Best score:", grid_search_cv.best_score_)

Best parameters: {'regressor__learning_rate': 0.3, 'regressor__max_depth': 5, 'regressor__n_estimators': 1500}
Best score: -0.25799729766452256


The accuracy is 0.25 which is not good enough.

### Model Saving and Loading

In [ ]:
# Save the model
joblib.dump(grid_search_cv, MODELS_PATH + 'XGBoost-CV.joblib')

['/content/drive/MyDrive/data-science-notebooks/rossmann-store-sales-prediction/models/XGBoost-CV.joblib']

In [ ]:
# Load the model
xgb_loaded = joblib.load(MODELS_PATH + 'XGBoost-CV.joblib')

## Neural Network

I plan to implement a Neural Netwrok with Entity Embedding to achieve a higher accuracy.

In [ ]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cuda


### Data loader


In [ ]:
batch_size = 64

# Data splitting
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=936)

# Convert Data Frame to Tensor
X_train_tensor = torch.tensor(X_train.values, dtype=torch.long)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.long)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)

# Load Tensor Dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Load Data Loader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print("DataLoader is ready!")

DataLoader is ready!


### Neural Network Architecture with Entity Embeddings

In [ ]:
# Count the number of categories for each column
num_cate = [len(X[col].unique()) for col in X.columns]

# Calculate the embedding dimension for each categorical feature
emb_dim = [min(50, round(np.sqrt(i))) for i in num_cate]

# Build the Neural Network Architecture
class MyNN(nn.Module):
    def __init__(self, num_cate, emb_dim, hidden_layers, dropouts):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(nc, dim) for nc, dim in zip(num_cate, emb_dim)])
        self.fc = nn.Sequential(
            nn.Linear(sum(emb_dim), hidden_layers[0]),
            nn.ReLU(),
            nn.Dropout(dropouts[0]),
            nn.Linear(hidden_layers[0], hidden_layers[1]),
            nn.ReLU(),
            nn.Dropout(dropouts[1]),
            nn.Linear(hidden_layers[1], 1)
        )

    def forward(self, x):
        x = [emb(x[:, i]) for i, emb in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        x = self.fc(x)
        return x

print("Neural Network is ready!")

Neural Network is ready!


**Build a validation help function**

In [ ]:
# Build a RMSPE score metric for cross validation
# Build a RMSPE score metric for cross validation
def rmspe_nn(y_true, y_pred):
    # Filter out entries where y_true is zero to avoid division by zero
    non_zero_mask = y_true != 0
    y_true_filtered = y_true[non_zero_mask]
    y_pred_filtered = y_pred[non_zero_mask]

    # Ensure predictions are non-negative
    y_pred_filtered = torch.clamp(y_pred_filtered, min=0.0)

    return torch.sqrt(torch.mean(torch.square((y_true_filtered - y_pred_filtered) / y_true_filtered)))

print("RMSPE scorer completed!")

RMSPE scorer completed!


In [ ]:
def val_func(model, val_dataloader, device):
    model.eval()
    val_rmspes_final = []
    with torch.no_grad():
        for x_batch, y_batch in val_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            y_pred = model(x_batch).squeeze()
            val_rmspe = rmspe_nn(y_batch, y_pred)
            val_rmspes_final.append(val_rmspe.item())

    avg_val_rmspe_final = np.mean(val_rmspes_final)

    return avg_val_rmspe_final

print("Validation function is ready!")

Validation function is ready!


### Model Training

In [ ]:
# Setting
hidden_layers = [2000, 800]
dropouts = [0.2, 0.2]
learning_rate = 0.01
epochs = 5
my_model = MyNN(num_cate=num_cate, emb_dim=emb_dim, hidden_layers=hidden_layers, dropouts=dropouts).to(device)
optimizer = optim.Adam(my_model.parameters(), lr=learning_rate)

# Define OneCycleLR
steps_per_epoch = len(train_dataloader)
scheduler = OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=steps_per_epoch, epochs=epochs)

In [ ]:
# Training loop
print(f"Starting training on device: {device}")
for epoch in range(epochs):
    my_model.train()
    train_losses = []
    for x_batch, y_batch in train_dataloader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = my_model(x_batch).squeeze()
        loss = rmspe_nn(y_batch, y_pred)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_losses.append(loss.item())

    avg_train_loss = np.mean(train_losses)
    avg_val_rmspe = val_func(my_model, val_dataloader, device)

    print(f"Epoch {epoch+1}/{epochs}, Training Loss (RMSPE): {avg_train_loss:.4f}, Validation RMSPE: {avg_val_rmspe:.4f}")

print("Training completed!")

Starting training on device: cuda
Epoch 1/5, Training Loss (RMSPE): 0.2211, Validation RMSPE: 0.1579
Epoch 2/5, Training Loss (RMSPE): 0.1608, Validation RMSPE: 0.1264
Epoch 3/5, Training Loss (RMSPE): 0.1475, Validation RMSPE: 0.1158
Epoch 4/5, Training Loss (RMSPE): 0.1336, Validation RMSPE: 0.1042
Epoch 5/5, Training Loss (RMSPE): 0.1251, Validation RMSPE: 0.1015
Training completed!


The RMSPE accuracy is 0.1015. It's pretty good.

### Model saving and loading

In [ ]:
# Save the model
torch.save(my_model.state_dict(), MODELS_PATH + 'NN.pth')

In [ ]:
# Load the model
nn_loaded = MyNN(num_cate=num_cate, emb_dim=emb_dim, hidden_layers=hidden_layers, dropouts=dropouts)
nn_loaded.load_state_dict(torch.load(MODELS_PATH + 'NN.pth'))

<All keys matched successfully>

## Business Analysis

Because nueral network is hard to interpret, I'll interpret the XGBoost model

In [ ]:
# Extract the model and feature names
xgb_best_pipeline = xgb_loaded.best_estimator_
xgb_regressor = xgb_best_pipeline.named_steps['regressor']
feature_importances = xgb_regressor.feature_importances_
preprocessor = xgb_best_pipeline.named_steps['preprocessor']
target_encoded_features = preprocessor.named_transformers_['target_encoding'].get_feature_names_out()
one_hot_encoded_features = preprocessor.named_transformers_['one_hot_encoding'].get_feature_names_out()

# Combine feature names in the correct order
all_feature_names = list(target_encoded_features) + list(one_hot_encoded_features)

# Create a mapping feature names to importance scores
feature_importance_series = pd.Series(feature_importances, index=all_feature_names)

# Display the top features
print("Top 10 Feature Importances:")
print(feature_importance_series.nlargest(10))


Top 10 Feature Importances:
DayOfWeek_6       0.325968
StateHoliday_3    0.144913
StateHoliday_2    0.129493
StateHoliday_4    0.097960
Store             0.017429
DayOfWeek_0       0.015582
Month_11          0.012897
Day_30            0.011607
Day_22            0.011082
StateHoliday_1    0.010423
dtype: float32


### Business Context
Accurate daily sales forecasting is important for Rossmann stores to operate efficiently and maximize profitability. It directly impacts several critical business functions:
-   **Inventory Management**: Precise forecasts prevent stockouts or overstocking, reducing waste and ensuring product availability.
-   **Staffing Optimization**: Understanding sales peaks and troughs allows for efficient scheduling of staff, preventing understaffing during busy periods and overstaffing during quiet times.
-   **Promotional Planning**: Forecasts help in evaluating the potential impact of promotions and planning their timing for maximum effect.
-   **Financial Planning**: Reliable sales predictions are essential for budget allocation and financial reporting.

### Key Features
Based on the XGBoost model's feature importance analysis, the following factors are identified as the most influential in predicting Rossmann store sales:

1.  **DayOfWeek_6 (Sunday)**: With a significantly high importance of `0.326`, Sunday is the most impactful feature. This strongly suggests that sales patterns on Sundays are distinctly different from other days.

2.  **StateHoliday (categories 2, 3, 4)**: StateHoliday categories, particularly `StateHoliday_3` (Easter holiday) with `0.145` and `StateHoliday_2` (public holiday) with `0.129`, and `StateHoliday_4` (Christmas) with `0.098` show very high importance. This is expected as state holidays significantly alter shopping behavior, often leading to store closures or altered operating hours, thus heavily impacting sales.

3.  **Store**: The `Store` feature (target encoded) has an importance of `0.017`. This indicates that individual store characteristics, captured by the target encoding, play a role in sales, implying that sales are not uniformly distributed across all stores and each store has its own unique sales profile.

Other notable features include:
-   **DayOfWeek_0 (Monday)**: `0.016`, suggesting Monday also has a distinct sales pattern.
-   **Month_11 (December)**: `0.013`, indicating higher sales towards the end of the year, likely due to holiday shopping.
-   **Day of Month**: Specific days of the month like `Day_30`, `Day_22` also show considerable importance, suggesting cyclical patterns within months.

These importances highlight the dominance of temporal and store-specific factors in driving sales.